============Quantum Kaizen============

              Version 0.1

     A game by the Decodoku Project

In [1]:
# hide code
# adapted from http://blog.nextgenetics.net/?e=102
from IPython.display import HTML
HTML('''<script>
showCode=true; 
$('div.input').hide();
</script>
The raw code for this IPython notebook is hidden when run''')

In [2]:
# Import Python interface for the web API
import sys
if sys.version_info < (3,0):
    raise Exception("Please use Python version 3 or greater.")
from IBMQuantumExperience import IBMQuantumExperience

In [3]:
# Configure API Token (needs a Qconfig.py file, see https://github.com/IBM/qiskit-sdk-py/blob/master/scripts/getting_started_with_the_qx_api.ipynb)
import Qconfig
api = IBMQuantumExperience.IBMQuantumExperience(Qconfig.APItoken, Qconfig.config)

In [4]:
# Import other things we'll need
import getpass, time

In [5]:
input("Press Enter to continue...\n")

Press Enter to continue...



''

In [ ]:
print("Quantum Yoshino is a simple version of Battleships, played on a quantum computer.\n")
print("Each player places three ships in five possible locations.\n")
print("Then they place bombs until one player loses all their ships.\n")
print("The first ship placed by each player takes 1 bomb to destroy.\n")
print("The second ship takes 2, and the third takes 3.\n")

Quantum Yoshino is a simple version of Battleships, played on a quantum computer.

Each player places three ships in five possible locations.

Then they place bombs until one player loses all their ships.

The first ship placed by each player takes 1 bomb to destroy.

The second ship takes 2, and the third takes 3.



In [ ]:
input("Press Enter to start placing ships...\n")

In [ ]:
# Now we get the players to give us positions of the ships
# The variable ship[X][Y] will hold the position of the Yth ship of player X+1
shipPos = [ [-1]*3 for _ in range(2)] # all values are initialized to the impossible position -1|

In [ ]:
# loop over both players and all three ships for each
for player in [0,1]:
    
    print("\n\n\nIt's now Player " + str(player+1) + "'s turn to choose positions for ships.\n")
    
    print("The numbers below mark places you can put a ship.\n")
    print("4       0")
    print("|\     /|")
    print("| \   / |")
    print("|  \ /  |")
    print("|   2   |")
    print("|  / \  |")
    print("| /   \ |")
    print("|/     \|")
    print("3       1\n")
    
    for ship in [0,1,2]:
        
        # ask for a position for each ship, and keep asking until a valid answer is given
        choosing = True
        while (choosing):

            # get player input
            position = getpass.getpass("Choose a position for ship " + str(ship+1) + " (0, 1, 2, 3 or 4)\n")

            # see if this is a valid input. ask for another if not
            if position.isdigit(): # valid answers  have to be integers
                position = int(position)
                if (position in [0,1,2,3,4]) and (not position in shipPos[player]): # they need to be between 0 and 5, and not used for another ship of the same player
                    shipPos[player][ship] = position
                    choosing = False
                    print ("\n")
                elif position in shipPos[player]:
                    print("\nYou already have a ship there. Try again.\n")
                else:
                    print("\nThat's not a valid position. Try again.\n")
            else:
                print("\nThat's not a valid position. Try again.\n")

In [ ]:
# Finally it's time for the main game loop.

# In this we ask players for where they want to bomb, apply the bombs and repeat until all ships are destroyed

# the game variable will be set to False once the game is over
game = True

# the variable bombs[X][Y] will hold the number of times position Y has been bombed by player X+1
bomb = [ [0]*5 for _ in range(2)] # all values are initialized to the impossible position -1|

while (game):
    
    input("Press Enter to place some bombs...\n")
    
    # ask both players where they want to bomb
    for player in [0,1]:
    
        print("\n\n\nIt's now Player " + str(player+1) + "'s turn.\n")

        # keep asking until a valid answer is given
        choosing = True
        while (choosing):

            # get player input
            print("Choose a position to bomb (0, 1, 2, 3 or 4)\n")
            print("4       0")
            print("|\     /|")
            print("| \   / |")
            print("|  \ /  |")
            print("|   2   |")
            print("|  / \  |")
            print("| /   \ |")
            print("|/     \|")
            print("3       1\n")
            position = input("")

            # see if this is a valid input. ask for another if not
            if position.isdigit(): # valid answers  have to be integers
                position = int(position)
                if position in [0,1,2,3,4]: # they need to be between 0 and 5, and not used for another ship of the same player
                    bomb[player][position] = bomb[player][position] + 1
                    choosing = False
                    print ("\n")
                else:
                    print("\nThat's not a valid position. Try again.\n")
            else:
                print("\nThat's not a valid position. Try again.\n")
                
    # now we write the OPENQASM scripts that will implement this on the quantum processor
    
    runGrid = ["",""]
    
    for player in range(2):
        # first all the preamble for a circuit with 5 qubits and 3 outputs
        runGrid[player] = """
OPENQASM 2.0;
include "qelib1.inc";
qreg q[5];
creg c[3];
        """
        # create ships with a Hadamard
        for ship in range(3):
            runGrid[player] = runGrid[player] + "\n" + "h q[" + str(shipPos[player][ship]) + "];"
        
        # add the bombs (of the opposing player)
        runGrid[player] = runGrid[player] + "\n"
        for position in range(5):
            # add as many bombs as have been placed at this position
            for n in range( bomb[(player+1)%2][position] ):
                # the effectiveness of the bomb (which means the quantum operation we apply) depends on which ship it is
                for ship in [0,1,2]:
                    if ( position == shipPos[player][ship] ):
                        frac = 1/(4-ship)
                        runGrid[player] = runGrid[player] + "\n" + "u1( " + str(frac) + " * pi) q[" + str(position) + "];"
                
        #finally, measure them (in X basis, so Hadamards first)
        runGrid[player] = runGrid[player] + "\n"
        for ship in range(3):
            runGrid[player] = runGrid[player] + "\n" + "h q[" + str(shipPos[player][ship]) + "];"
        runGrid[player] = runGrid[player] + "\n"
        for ship in range(3):
            runGrid[player] = runGrid[player] + "\n" + "measure q[" + str(shipPos[player][ship]) + "] -> c[" + str(ship) + "];"
    
    # to see what the quantum computer is asked to do, we can print these files
    # print(runGrid[0])
 # for debugging    # print(runGrid[1])
 # for debugging    print("\n")
    
    # let's get these jobs running
    out = api.run_job(qasms = [{'qasm' : runGrid[0] },{'qasm' : runGrid[1] }],device = 'sim',shots = 1024, max_credits=3)
    print(out['status'])

    # keep the players updated on status
    jobids=out['id']
    results = api.get_job(jobids)
    print(results['status'])
    while (results['status'] == 'RUNNING'):
        time.sleep(2)
        results = api.get_job(jobids)
        print(results['status'])
    print ("\n")
    
    # get data
    get_data = lambda results, i: results['qasms'][i]['result']['data']['counts']
    
    grid = [ get_data(results,0) , get_data(results,1) ]
    
    # get damage for all ships and see if the game is over
    damage = [ [0]*3 for _ in range(2)]
    # extracted from the results in grid
    for player in [0,1]:
        for r1 in [0,1]:
            for r2 in [0,1]:

                    # results are stored in grid[player] as a dictionary
                    # keys are bit strings, with the leftmost bit corresponding to the first ship
                    # for each ship we construct a string with a 1 (which corresponds to damage) in the relevant position
                    bits0 = str(r2) + str(r1) + "1"
                    bits1 = str(r2) + "1" + str(r1)
                    bits2 = "1" + str(r2) + str(r1)

                    if bits0 in grid[player].keys():
                        damage[player][0] = damage[player][0] + grid[player][bits0]/1024
                    if bits1 in grid[player].keys():
                        damage[player][1] = damage[player][1] + grid[player][bits1]/1024
                    if bits2 in grid[player].keys():  
                        damage[player][2] = damage[player][2] + grid[player][bits2]/1024
         
        # give results to players
        input("Press Enter to see the results for Player " + str(player+1) + "'s ships...\n")
        
        # report the ships with significant damange
        # ideally this would be non-zero damage, but noise means that can happen for ships that haven't been hit
        # so we choose 5% as the threshold
        display = [" ? "]*5
        for ship in [0,1,2]:
            if ( damage[player][ship] > 0.05 ):
                display[ shipPos[player][ship] ] = str(int( 100*damage[player][ship] )) +"%"
        
        print("Here is the percentage damage for ships that have been bombed.\n")
        print(display[ 4 ] + "     " + display[ 0 ])
        print(" |\     /|")
        print(" | \   / |")
        print(" |  \ /  |")
        print(" |  " + display[ 2 ] + "  |")
        print(" |  / \  |")
        print(" | /   \ |")
        print(" |/     \|")
        print(display[ 3 ] + "     " + display[ 1 ])
        print("\n")
        print("Ships with 95% damage or more have been destroyed\n")
        
        print("\n")
           
        # if a player has all their ships destroyed, the game is over
        # ideally this would mean 100% damage, but we go for 95% because of noise again
        if (damage[player][0]>.95) and (damage[player][1]>.95) and (damage[player][2]>.95):
            print ("All your ships have been destroyed!\n")
            game = False
            
    if (game is False):
        print ("Game Over\n")
        